In [1]:
import pyspark.sql.functions as f_
from pyspark.sql.window import Window
import sys
sys.path.append('/lakehouse/default/Files/GENERALES')
import f_amr

StatementMeta(, ff1a4964-1d7d-4252-b306-8393b1a42f21, 3, Finished, Available, Finished)

#### Adaptación de variables respecto a BDD

In [2]:
#Curva de intentos
df_c3 = spark.read.format("delta").load("abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/258fe07c-f858-473a-9bb3-706d3b0a0475/Tables/BDD_TLMK/curvas_p3")

#Seleccionar unicamente lo tocado en base a las fechas
df_c3 = df_c3.withColumn("TOQUE_DIA", f_.when(f_.col("FECHA_INICIO") >= f_.col("FECHA"), 1).otherwise(0)).filter(f_.col("TOQUE_DIA") == 1)

# Ventanas
window_ciclo = Window.partitionBy("CICLO", "NO_PERSONA").orderBy("FECHA_INICIO")
window_dia = Window.partitionBy("FECHA_INICIO", "NO_PERSONA").orderBy("FECHA_INICIO")
window_semana = Window.partitionBy("NO_PERSONA", "SEMANA_VENTA").orderBy("FECHA_INICIO")

# Modificar AGRUPADOR_ROBOT y DESCRIPCION
df_modificado = df_c3.withColumns({"AGRUPADOR_ROBOT": f_.when(f_.col("ULTIMO_RESULTADO_AGENTE_TEXTO").isNotNull(),"LLAMADA CONTESTADA").otherwise(f_.col("AGRUPADOR_ROBOT")),
                                   "DESCRIPCION": f_.when(f_.col("ULTIMO_RESULTADO_AGENTE_TEXTO").isNotNull(),"LLAMADA CONTESTADA").otherwise(f_.col("DESCRIPCION"))
                                  })

# Calcular valores únicos 
df_final = df_modificado.withColumns({
    "REG_UNICOS_CICLO": f_.when(f_.row_number().over(window_ciclo) == 1, 1).otherwise(0),
    "REG_UNICOS_DIA": f_.when(f_.row_number().over(window_dia) == 1, 1).otherwise(0),
    "REG_UNICOS_SEM": f_.when(f_.row_number().over(window_semana) == 1, 1).otherwise(0)
})

# Calcular métricas de llamadas

# Preparamos la columna limpia para usarla varias veces
col_tip = f_.trim(f_.lower(f_.col("TIPIFICACION")))
col_desc = f_.trim(f_.lower(f_.col("DESCRIPCION")))

df_final = df_final.withColumns({"CONTESTADAS": f_.when(col_desc == "llamada contestada", f_.col("CONTADOR")).otherwise(0),
                                 "EFECTIVAS": f_.when((~col_tip.contains("no contesta")) & (~col_tip.contains("cort")) & (~col_tip.contains("contestadora")), f_.col("CONTESTADAS")).otherwise(0),
                                 "NO_EFECTIVAS": f_.when(col_tip.contains("no contesta") | col_tip.contains("cort") | col_tip.contains("contestadora"), f_.col("CONTESTADAS")).otherwise(0),
                                 "CORTADAS": f_.when(col_tip.contains("no contesta") | col_tip.contains("cort"), f_.col("CONTESTADAS")).otherwise(0),}).drop("RSP_ESCUELA_OK", "MEDIUM", "ESTADO", "BANNER")
# Adición de variables

df_sitio = spark.read.format("delta").load("abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/258fe07c-f858-473a-9bb3-706d3b0a0475/Tables/BDD_TLMK/agrupador_sitio")
df_sitio = df_sitio.withColumnRenamed('CICLO', 'CICLO2')

df_pros = spark.read.format("delta").load("abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/258fe07c-f858-473a-9bb3-706d3b0a0475/Tables/BDD_TLMK/registros_prospeccion")
df_pros = df_pros.withColumnRenamed('CICLO', 'CICLO2')

# JOIN con base prospeccion
df_join = (df_final.join( df_pros, (df_final["NO_PERSONA"] == df_pros["no_de_persona"]) & (df_final["CICLO"] == df_pros["CICLO2"]), "left")
).drop('AGRUPADOR_SITIO','BANNER','MEDIUM','ESTADO')
df_join = df_join.dropDuplicates(["RECORDID"])

# JOIN con base digital
df_join2 = (df_join.join(df_sitio,(df_join["NO_PERSONA"] == df_sitio["no_de_persona"]) & (df_join["CICLO"] == df_sitio["CICLO2"]),"left")
).drop('NO_DE_PERSONA','CICLO2')
df_join2 = df_join2.dropDuplicates(["RECORDID"])

# Agrupación final
columnas_agrupacion = [
    "AGRUPADOR_ROBOT",
    "AGRUPADOR_TIPIFICACION",
    "CAMPUS",
    "CARRERA",
    "CICLO",
    "CICLO_LAUREATE",
    "FUENTE",
    "FUENTE_AGRUPADA",
    "CLUSTER_TMK",
    "LINEA_NEGOCIO_ING",
    'TIPO_TMK',
    "TIPIFICACION",
    "TIPO_DE_PROGRAMA",
    "TIPO_DE_INGRESO",
    "TIPO_RECICLADO",
    'TOQUE_DIA',
    "VARIABLE",
    "DESCRIPCION",
    'NO_PERSONA',
    "DIA",
    "FECHA",
    "FECHA_INICIO",
    "HOMOLOGADO",
    "OFERTA_EDUCATIVA",
    "OPTIMO_MARCABLE",
    "POTENCIAL",
    "REGISTROS_TOCADOS",
    "SEMANA_LEAD",
    "SEMANA_VENTA",
    'AGRUPADOR_SITIO',
    "ULTIMO_RESULTADO_AGENTE_TEXTO"
]

df_agrupado = df_join2.groupBy(columnas_agrupacion).agg(
    f_.sum("REG_UNICOS_CICLO").alias("REG_UNICOS_CICLO"),
    f_.sum("REG_UNICOS_DIA").alias("REG_UNICOS_DIA"),
    f_.sum("REG_UNICOS_SEM").alias("REG_UNICOS_SEM"),
    f_.sum("CONTADOR").alias("INTENTOS"),
    f_.sum("CONTESTADAS").alias("CONTESTADAS"),
    f_.sum("CORTADAS").alias("CORTADAS"),
    f_.sum("EFECTIVAS").alias("EFECTIVAS"),
    f_.sum("NO_EFECTIVAS").alias("NO_EFECTIVAS")
)

df_modificado = df_agrupado.withColumns({"AGRUPADOR_ROBOT": f_.when(f_.col("ULTIMO_RESULTADO_AGENTE_TEXTO").isNotNull(), "LLAMADA CONTESTADA").otherwise(f_.col("AGRUPADOR_ROBOT")),
                                         "DESCRIPCION": f_.when(f_.col("ULTIMO_RESULTADO_AGENTE_TEXTO").isNotNull(), "LLAMADA CONTESTADA").otherwise(f_.col("DESCRIPCION"))
                                        })

StatementMeta(, ff1a4964-1d7d-4252-b306-8393b1a42f21, 4, Finished, Available, Finished)

#### Normalización básica proyecto score

###### Selección de columnas iniciales

In [3]:
#Fuente
curva_bdd_unitec = df_modificado #spark.read.format('delta').load('')
# Limpieza de columnas
seleccion_1 = \
{'CAMPUS':'CAMPUS',
'CARRERA':'CARRERA',
'CICLO':'CICLO',
'CLUSTER_TMK':'LINA_H_CLUSTER',
'CONTESTADAS':'CONTESTADAS',
'CORTADAS':'CORTADAS',
'DESCRIPCION':'DESCRIPCION',
'EFECTIVAS':'EFECTIVAS',
'FECHA':'FECHA',
'FECHA_INICIO':'FECHA_INICIO',
'FUENTE':'FUENTE',
'FUENTE_AGRUPADA':'FUENTE_AGRUPADA',
'INTENTOS':'INTENTOS',
'LINEA_NEGOCIO_ING':'LINEA_NEGOCIO_ING',
'NO_EFECTIVAS':'NO_EFECTIVAS',
'NO_PERSONA':'NO_DE_PERSONA',
'REG_UNICOS_CICLO':'REG_UNICOS_CICLO',
'REG_UNICOS_DIA':'REG_UNICOS_DIA',
'REG_UNICOS_SEM':'REG_UNICOS_SEM',
'TIPIFICACION':'TIPIFICACION',
'TIPO_DE_INGRESO':'TIPO_DE_INGRESO',
'TIPO_DE_PROGRAMA':'TIPO_DE_PROGRAMA',
'TIPO_RECICLADO':'TIPO_RECICLADO',
'TIPO_TMK':'TEAM_TMK',
'TOQUE_DIA':'TOQUE_DIA',
'VARIABLE':'VARIABLE'}

c = {'25-2': 'C1','25-3': 'C2','26-1': 'C3'}

curva_bdd_unitec = f_amr.select_cols(curva_bdd_unitec, seleccion_1)
curva_bdd_unitec = f_amr.map_cols(curva_bdd_unitec,'CICLO',c,'CICLO_LAUREATE')
curva_bdd_unitec = curva_bdd_unitec.withColumn('RECICLADO', f_.when(f_.col('TIPO_RECICLADO').startswith('R'), 'RECICLADO').otherwise('NUEVO'))
curva_bdd_unitec = curva_bdd_unitec.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'FECHA_INICIO')))


StatementMeta(, ff1a4964-1d7d-4252-b306-8393b1a42f21, 5, Finished, Available, Finished)

###### Adición y corrección de columnas básicas KPI

- Catálogos

In [4]:
#Fuente
funnel_unitec_a = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/ANALYTIKUS/FUNNEL_UNITEC_A.parquet')
funnel_unitec_a = funnel_unitec_a.withColumn('FECHA', f_.to_date('FECHA', "yyyy-MM-dd"))
#Limpieza de columnas
seleccion_2 = \
{'CARRERA':'CARRERA',
'CICLO':'CICLO',
'FECHA':'FECHA',
'NO_DE_PERSONA':'NO_DE_PERSONA',
'AGRUPADOR_SITIO':'AGRUPADOR_SITIO',
'LINEA_H':'LINEA_H'}

funnel_unitec_a = f_amr.select_cols(funnel_unitec_a, seleccion_2)
funnel_unitec_a = f_amr.uniques(funnel_unitec_a, ['NO_DE_PERSONA', 'CICLO'], [('NO_DE_PERSONA', 'asc'),('FECHA', 'desc')])
cat_carrera = funnel_unitec_a.select('CARRERA', 'LINEA_H').distinct()

StatementMeta(, ff1a4964-1d7d-4252-b306-8393b1a42f21, 6, Finished, Available, Finished)

- Ajuste columnas KPI

In [5]:
# Adición selectiva
match_Agrcl_1 = curva_bdd_unitec.drop('CARRERA').join(funnel_unitec_a.select('NO_DE_PERSONA', 'CICLO', 'CARRERA', 'AGRUPADOR_SITIO', 'LINEA_H'),['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_Agrcl_left_1 = curva_bdd_unitec.join(funnel_unitec_a,['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_Agrcl_right_1 = funnel_unitec_a.join(curva_bdd_unitec,['NO_DE_PERSONA','CICLO'], 'left_anti')

match_Agrcl_2 = no_match_Agrcl_left_1.drop('CARRERA').join(no_match_Agrcl_right_1.select('NO_DE_PERSONA', 'CARRERA', 'AGRUPADOR_SITIO', 'LINEA_H'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])
no_match_Agrcl_left_2 = no_match_Agrcl_left_1.join(no_match_Agrcl_right_1, 'NO_DE_PERSONA', 'left_anti')
match_Agrcl_3 = no_match_Agrcl_left_2.join(cat_carrera, 'CARRERA', 'left').dropDuplicates(['INDICE'])

curva_bdd_unitec = match_Agrcl_1.unionByName(match_Agrcl_2, allowMissingColumns=True).unionByName(match_Agrcl_3, allowMissingColumns=True)

# Normalización post uniones
curva_bdd_unitec = curva_bdd_unitec.withColumns({'ESTATUS_MARCACION': f_.lit('MARCADO'),
                                                 'AGRUPADOR_SITIO': f_.when(f_.col('AGRUPADOR_SITIO').isNull(), 'NO ES MARKETING').otherwise(f_.col('AGRUPADOR_SITIO'))})

seleccion_3= \
{'AGRUPADOR_SITIO':'AGRUPADOR_SITIO',
'CAMPUS':'CAMPUS',
'CARRERA':'CARRERA',
'CICLO':'CICLO',
'CICLO_LAUREATE':'CICLO_LAUREATE',
'ESTATUS_MARCACION':'ESTATUS_MARCACION',
'FECHA_INICIO':'FECHA',
'FUENTE':'FUENTE',
'FUENTE_AGRUPADA':'FUENTE_AGRUPADA',
'INDICE':'INDICE',
'INTENTOS':'INTENTOS',
'LINA_H_CLUSTER':'LINA_H_CLUSTER',
'LINEA_H':'LINEA_H',
'LINEA_NEGOCIO_ING':'LINEA_NEGOCIO_ING',
'NO_DE_PERSONA':'NO_DE_PERSONA',
'NO_EFECTIVAS':'NO_EFECTIVAS',
'RECICLADO':'RECICLADO',
'REG_UNICOS_CICLO':'REG_UNICOS_CICLO',
'REG_UNICOS_DIA':'REG_UNICOS_DIA',
'REG_UNICOS_SEM':'REG_UNICOS_SEM',
'TEAM_TMK':'TEAM_TMK',
'TIPO_DE_PROGRAMA':'TIPO_DE_PROGRAMA',
'TIPO_DE_INGRESO':'TIPO_DE_INGRESO',
'VARIABLE':'VARIABLE'}

curva_bdd_unitec = f_amr.select_cols(curva_bdd_unitec, seleccion_3)

# Adición de Semana
calendario = spark.read.option('header', True).option('inferSchema', True).csv('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/ANALYTIKUS/CALENDARIO.csv')
calendario = calendario.withColumn('FECHA', f_.to_date('FECHA', 'dd/MM/yyyy'))
calendario = f_amr.cast_cols(calendario, {'SEMANA':'int'})

curva_bdd_unitec = curva_bdd_unitec.join(calendario.select('FECHA', 'SEMANA'), 'FECHA', 'inner').drop('INDICE')

StatementMeta(, ff1a4964-1d7d-4252-b306-8393b1a42f21, 7, Finished, Available, Finished)

In [6]:
from pyspark.sql.functions import current_timestamp
curva_bdd_unitec = curva_bdd_unitec.withColumn("ACTUALIZACION", f_.expr("current_timestamp() - INTERVAL 6 HOURS"))
#curva_bdd_unitec.repartition(8).write.option("overwriteSchema", "true").format("delta").mode("overwrite").saveAsTable("CURVAS.CURVA_INTENTOS_UNITEC")
p1= 'abfss://8c61db5f-46e1-4c72-8d6f-648bf05cacac@onelake.dfs.fabric.microsoft.com/0d2e9829-8a43-4592-a9d7-db37f7032a1c/Files/BASES_EN_PROCESO/CURVA_INTENTOS_UNITEC'
curva_bdd_unitec.repartition(10).write.mode("overwrite").parquet(p1)

StatementMeta(, ff1a4964-1d7d-4252-b306-8393b1a42f21, 8, Submitted, Running, Running)